In [3]:
import requests

all_data = []

url = "https://nypdonline.org/api/reports/b805fa11-d5d2-43f7-8c23-1649f5d387f1/data"
headers = {
    "Content-Type": "application/json",
    "Referer": "https://nypdonline.org/link/officer-profile"
}

for letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
    payload = [
        {"key": "@SearchName", "values": [""]},
        {"key": "@LastNameFirstLetter", "values": [letter]}
    ]

    response = requests.post(url, headers=headers, json=payload)
    
    if response.status_code == 200:
        data = response.json()
        all_data.extend(data) 

        print(f"Fetching letter: {letter}")
        print(f"Yup Got {len(data)} officers for {letter}")
    
    else:
        print(f"❌ Failed on {letter} — Status: {response.status_code}")


Fetching letter: A
Yup Got 2069 officers for A
Fetching letter: B
Yup Got 2386 officers for B
Fetching letter: C
Yup Got 3298 officers for C
Fetching letter: D
Yup Got 2013 officers for D
Fetching letter: E
Yup Got 587 officers for E
Fetching letter: F
Yup Got 1316 officers for F
Fetching letter: G
Yup Got 2141 officers for G
Fetching letter: H
Yup Got 1579 officers for H
Fetching letter: I
Yup Got 320 officers for I
Fetching letter: J
Yup Got 851 officers for J
Fetching letter: K
Yup Got 1178 officers for K
Fetching letter: L
Yup Got 1861 officers for L
Fetching letter: M
Yup Got 3710 officers for M
Fetching letter: N
Yup Got 731 officers for N
Fetching letter: O
Yup Got 702 officers for O
Fetching letter: P
Yup Got 2056 officers for P
Fetching letter: Q
Yup Got 150 officers for Q
Fetching letter: R
Yup Got 2580 officers for R
Fetching letter: S
Yup Got 3121 officers for S
Fetching letter: T
Yup Got 1242 officers for T
Fetching letter: U
Yup Got 157 officers for U
Fetching letter: V
Y

In [5]:
type(all_data)

list

In [6]:
cleaned_rows = []

for officer in all_data:  # assuming 'data' is your list of officers
    cols = officer["columns"]
    row = {
        "TAXID": officer["filterRowValue"],
        "Name": cols[0]["value"].strip(),
        "Rank": cols[1]["value"].strip(),
        "Shield": cols[2]["value"].strip(),
        "Hire Date": cols[3]["value"].strip(),
        "Arrests Total": cols[4]["value"].strip(),
        "Recognitions": cols[5]["value"].strip()
    }
    cleaned_rows.append(row)

In [7]:
import pandas as pd
df = pd.DataFrame(cleaned_rows)
df

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,TAXID,Name,Rank,Shield,Hire Date,Arrests Total,Recognitions
0,959433,"AANOUZ, ABDELHADI",DETECTIVE SPECIALIST,5664,10/7/2015 12:00:00 AM,104,0
1,975305,"AARONS, CAMELIA M",POLICE OFFICER,29755,10/19/2022 12:00:00 AM,27,0
2,941300,"AARONSON, JACOB J",POLICE OFFICER,44,7/10/2006 12:00:00 AM,154,5
3,982069,"ABABIO, MALIK K",POLICE OFFICER,,4/30/2025 12:00:00 AM,0,0
4,972490,"ABAD, BRENDALIZ",POLICE OFFICER,5985,10/12/2021 12:00:00 AM,0,0
...,...,...,...,...,...,...,...
36668,954443,"ZVONIK, SERGEY A",POLICE OFFICER,30494,1/9/2013 12:00:00 AM,59,0
36669,933512,"ZWEBEN, HOWARD E",LIEUTENANT,,7/1/2003 12:00:00 AM,127,34
36670,961483,"ZWEIFLER, MICHAEL G",DETECTIVE SPECIALIST,4650,1/6/2016 12:00:00 AM,135,52
36671,935452,"ZWILLING, CHARLES J",SERGEANT,2777,7/1/2004 12:00:00 AM,227,3


In [ ]:
#export cell
df.to_csv("nypd_officers_full.csv", index=False)

Now for the extra information i gotta scrape

In [ ]:
import requests
import json
# test for json response structure
url = "https://nypdonline.org/api/reports/e59337f8-014e-46a0-af99-7b6a3ef4dfa3/data"
headers = {
    "Content-Type": "application/json",
    "Referer": "https://nypdonline.org/link/officer-profile"
}

payload = [
    {"key": "@TAXID", "values": ["941300"]}
]

response = requests.post(url, headers=headers, json=payload)

print(response.status_code)
print(json.dumps(response.json(), indent=2))

200
[
  {
    "filterKey": "@TAXID",
    "columns": [
      {
        "value": "11/14/2022 12:00:00 AM"
      },
      {
        "value": "0"
      },
      {
        "filterValue": "ALLEG941300",
        "value": "1"
      }
    ]
  }
]


In [ ]:
#this code works but it takes too long to run for all officers
headers = {
    "Content-Type": "application/json",
    "Referer": "https://nypdonline.org/link/officer-profile"
}

tabs = {
    "Arrests Processed": "b9f649fc-b33f-47b2-9d7e-a7760090ae39",
    "Recognition and Awards": "916ef68f-3d47-46be-8e8a-79567a3e61f9",
    "Disciplinary History": "e59337f8-014e-46a0-af99-7b6a3ef4dfa3"
}

detailed_data = []

for i, row in df.iterrows():
    taxid = str(row["TAXID"])
    officer_details = {"TAXID": taxid}

    for tab, tab_id in tabs.items():
        url = f"https://nypdonline.org/api/reports/{tab_id}/data"
        payload = [{"key": "@TAXID", "values": [taxid]}]

        try:
            response = requests.post(url, headers=headers, json=payload)
            if response.status_code == 200:
                data = response.json()

                # You can extract specific fields here:
                officer_details[f"{tab} Count"] = len(data)
            else:
                officer_details[f"{tab} Count"] = f"ERR:{response.status_code}"

        except Exception as e:
            officer_details[f"{tab} Count"] = f"EXC:{str(e)}"

        time.sleep(0.3)  # polite scraping

    detailed_data.append(officer_details)

KeyboardInterrupt: 

In [8]:
import concurrent.futures

# API headers and tab endpoints
headers = {
    "Content-Type": "application/json",
    "Referer": "https://nypdonline.org/link/officer-profile",
    "User-Agent": "Mozilla/5.0"
}

tabs = {
    "Arrests Processed": "b9f649fc-b33f-47b2-9d7e-a7760090ae39",
    "Recognition and Awards": "916ef68f-3d47-46be-8e8a-79567a3e61f9",
    "Disciplinary History": "e59337f8-014e-46a0-af99-7b6a3ef4dfa3"
}

# Refactored function: fetches all 3 tab counts for one officer
def get_officer_details(taxid):
    details = {"TAXID": taxid}
    for tab, tab_id in tabs.items():
        url = f"https://nypdonline.org/api/reports/{tab_id}/data"
        payload = [{"key": "@TAXID", "values": [taxid]}]

        try:
            response = requests.post(url, headers=headers, json=payload, timeout=10)
            if response.status_code == 200:
                data = response.json()
                details[f"{tab} Count"] = len(data)
            else:
                details[f"{tab} Count"] = f"ERR:{response.status_code}"
        except Exception as e:
            details[f"{tab} Count"] = f"EXC:{str(e)}"

        time.sleep(0.1)  # Light pause between tab requests

    return details

# Run the multithreading loop
results = []

with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    futures = [executor.submit(get_officer_details, str(row["TAXID"])) for _, row in df.iterrows()]

    for i, future in enumerate(concurrent.futures.as_completed(futures)):
        result = future.result()
        results.append(result)

        # Save progress every 1000 officers
        if (i + 1) % 1000 == 0:
            print(f"✅ Saved {i+1} officers")
            pd.DataFrame(results).to_csv("officer_profiles_partial.csv", index=False)

# Final save
df_extra= pd.DataFrame(results)
print("✅ Done scraping and saving!")


✅ Saved 1000 officers
✅ Saved 2000 officers
✅ Saved 3000 officers
✅ Saved 4000 officers
✅ Saved 5000 officers
✅ Saved 6000 officers
✅ Saved 7000 officers
✅ Saved 8000 officers
✅ Saved 9000 officers
✅ Saved 10000 officers
✅ Saved 11000 officers
✅ Saved 12000 officers
✅ Saved 13000 officers
✅ Saved 14000 officers
✅ Saved 15000 officers
✅ Saved 16000 officers
✅ Saved 17000 officers
✅ Saved 18000 officers
✅ Saved 19000 officers
✅ Saved 20000 officers
✅ Saved 21000 officers
✅ Saved 22000 officers
✅ Saved 23000 officers
✅ Saved 24000 officers
✅ Saved 25000 officers
✅ Saved 26000 officers
✅ Saved 27000 officers
✅ Saved 28000 officers
✅ Saved 29000 officers
✅ Saved 30000 officers
✅ Saved 31000 officers
✅ Saved 32000 officers
✅ Saved 33000 officers
✅ Saved 34000 officers
✅ Saved 35000 officers
✅ Saved 36000 officers
✅ Done scraping and saving!


In [10]:
df_extra.head()

,TAXID,Arrests Processed Count,Recognition and Awards Count,Disciplinary History Count
0,941301,4,1,0
1,979015,2,0,0
2,973879,3,1,0
3,972491,2,0,0
4,947962,3,11,0


In [19]:
final_df = df.merge(df_extra, on="TAXID", how="left")

In [21]:
final_df

,TAXID,Name,Rank,Shield,Hire Date,Arrests Total,Recognitions,Arrests Processed Count,Recognition and Awards Count,Disciplinary History Count
0,959433,"AANOUZ, ABDELHADI",DETECTIVE SPECIALIST,5664,10/7/2015 12:00:00 AM,104,0,4,0,0
1,975305,"AARONS, CAMELIA M",POLICE OFFICER,29755,10/19/2022 12:00:00 AM,27,0,2,0,0
2,941300,"AARONSON, JACOB J",POLICE OFFICER,44,7/10/2006 12:00:00 AM,154,5,4,5,1
3,982069,"ABABIO, MALIK K",POLICE OFFICER,,4/30/2025 12:00:00 AM,0,0,0,0,0
4,972490,"ABAD, BRENDALIZ",POLICE OFFICER,5985,10/12/2021 12:00:00 AM,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
36668,954443,"ZVONIK, SERGEY A",POLICE OFFICER,30494,1/9/2013 12:00:00 AM,59,0,4,0,0
36669,933512,"ZWEBEN, HOWARD E",LIEUTENANT,,7/1/2003 12:00:00 AM,127,34,3,34,0
36670,961483,"ZWEIFLER, MICHAEL G",DETECTIVE SPECIALIST,4650,1/6/2016 12:00:00 AM,135,52,4,52,2
36671,935452,"ZWILLING, CHARLES J",SERGEANT,2777,7/1/2004 12:00:00 AM,227,3,5,3,0


In [ ]:
detail_df = pd.DataFrame(detailed_data)

# Merge with original
full_df = df.merge(detail_df, on="TAXID", how="left")
hbop12erd
# Save
full_df.to_csv("nypd_officers_extra.csv", index=False